In [24]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import math
import mediapipe as mp

In [25]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [26]:
# 포즈 감지 함수
def detectPose(image,pose,display=True):
    # input 이미지 복사
    output_image = image.copy()
    # RGB로 변환
    imageRGB = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    # 렌드마크 찍기
    results = pose.process(imageRGB)
    # 사진 사이즈 구하기
    height, width, _ = image.shape
    # 렌드마크 값 리스트 만들기
    landmarks = []
    visible = []
    if results.pose_landmarks:
        # 렌드마크 그리기
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks, connections=mp_pose.POSE_CONNECTIONS)
        #렌드마크 정규값 landmarks 리스트에 넣기
        for landmark in results.pose_landmarks.landmark:
            landmarks.append((int(landmark.x*width), int(landmark.y*height), (landmark.z*width)))
            visible.append(landmark.visibility)
    # 사진 결과 보기
    if display:
        plt.figure(figsize=[22,22])
        plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Original Image");plt.axis('off');
        plt.subplot(122);plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        # 3차원으로도 봐보기
        mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
    else:
        return output_image, landmarks , visible

In [27]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS)

In [28]:
def extract_landmarks(results):
    poses_landmarks = np.array([[res.x,res.y,res.z,res.visibility]for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    return poses_landmarks

In [29]:
# 각도 구하는 함수
def calculateAngle(landmark1, landmark2, landmark3):
    x1,y1,_ = landmark1 # z 좌표는 사용 x
    x2,y2,_ = landmark2
    x3,y3,_ = landmark3
    # 3점의 각도 -> 선 2개 -> 각도
    angle = math.degrees(math.atan2(y3-y2, x3-x2) - math.atan2(y1-y2,x1-x2))

    angle = np.abs(angle)
    # 음수의 각도가 나오면 양수로 바꾸기
    if angle >180.0:
        angle = 360 - angle
        
    return angle

In [30]:
# 중앙 각도 구하는 함수
def center_Landmark(landmarks1,landmarks2):
    x1,y1,_ = landmarks1
    x2,y2,_ = landmarks2
    x_cen = (x1+x2)/2
    y_cen = (y1+y2)/2
    return x_cen,y_cen,_

In [31]:
# 데이터를 저장할 경로
data_path = os.path.join('LSTM_PIC')
# landmark를 저장할 경로
data_key = os.path.join('LSTM_DATAS')
# 감지해야 할 액션
actions = np.array(['bend', 'headup', 'right', 'waistup'])
# 30게의 비디오 -> 동영상의 수만큼 수집할 예정
no_sequences = 10

# 동영상의 길이 - 30프레임
sequences_length = 30

# 30개의 동영상  * 30프레임 * 132개의 키포인트

In [32]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            #os.makedirs(os.path.join(data_path,action,str(sequence)))
            os.makedirs(os.path.join(data_key,action,str(sequence)))
        except:
            pass

In [33]:
# 이미 있는 사진으로 data 만들기 
#no_sequences = 10 -> 폴더수  sequences_length = 30 -> 사진수
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequences_length):
                print(f'./LSTM_PIC/{action}/{sequence}/'+os.listdir(f'./LSTM_PIC/{action}/{sequence}')[frame_num])
                frame = cv2.imread(f'./LSTM_PIC/{action}/{sequence}/'+os.listdir(f'./LSTM_PIC/{action}/{sequence}')[frame_num])
                
                image, landmarks, visible = detectPose(frame, pose, display=False)
                if landmarks:
                    # 랜드마크 추출 후 각도 계산
                    # visible 가져오기
                    left_shoulder = visible[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    right_shoulder = visible[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    left_hip = visible[mp_pose.PoseLandmark.LEFT_HIP.value]
                    right_hip = visible[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    left_knee = visible[mp_pose.PoseLandmark.LEFT_KNEE.value]
                    right_knee = visible[mp_pose.PoseLandmark.RIGHT_KNEE.value]
                    left_ankle = visible[mp_pose.PoseLandmark.LEFT_ANKLE.value]
                    right_ankle = visible[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
                    
                    # 어깨 - 엉덩이 - 무릎 => 올리는 동작 각도 구하기 위함 -> Main angle
                    if left_hip>right_hip:
                        main_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                                            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                                            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value])
                    else:
                        main_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                                            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                                            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value])                                                    
                    
                    # 엉덩이 - 무릎 - 발목 => 무릎 꺾였는지 보기 위함
                    if left_knee > right_knee:
                        knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                                        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                                        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])
                    else:
                        knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                                        landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                                        landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
                                                        
                    # 어깨 허리  중앙값
                    cen_shoulder = center_Landmark(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value])
                    cen_hip = center_Landmark(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value])
                
                    # 코 - 어깨 - 엉덩이 => 목 각도
                    head_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.NOSE.value],
                                                cen_shoulder,cen_hip)

                    # 엉덩이 중앙의 y좌표와 중앙어깨의 y 좌표사이의 거리 
                    waist_len = cen_hip[1]-cen_shoulder[1]
                
                    globals()[f'f{action,frame_num}'] = knee_angle,head_angle,waist_len
                
                    npy_path = os.path.join(data_key, action, str(sequence), str(f'f{action,frame_num}'))
                    # npy_path -> ./MP_Data/action(right,headup,leg)/0~10/0~30
                    np.save(npy_path, globals()[f'f{action,frame_num}'])

            
cv2.destroyAllWindows()

            
                

./LSTM_PIC/bend/0/E-1 (1).jpg
./LSTM_PIC/bend/0/E-1 (10).jpg
./LSTM_PIC/bend/0/E-1 (11).jpg
./LSTM_PIC/bend/0/E-1 (12).jpg
./LSTM_PIC/bend/0/E-1 (13).jpg
./LSTM_PIC/bend/0/E-1 (14).jpg
./LSTM_PIC/bend/0/E-1 (15).jpg
./LSTM_PIC/bend/0/E-1 (16).jpg
./LSTM_PIC/bend/0/E-1 (17).jpg
./LSTM_PIC/bend/0/E-1 (18).jpg
./LSTM_PIC/bend/0/E-1 (19).jpg
./LSTM_PIC/bend/0/E-1 (2).jpg
./LSTM_PIC/bend/0/E-1 (20).jpg
./LSTM_PIC/bend/0/E-1 (21).jpg
./LSTM_PIC/bend/0/E-1 (22).jpg
./LSTM_PIC/bend/0/E-1 (23).jpg
./LSTM_PIC/bend/0/E-1 (24).jpg
./LSTM_PIC/bend/0/E-1 (25).jpg
./LSTM_PIC/bend/0/E-1 (26).jpg
./LSTM_PIC/bend/0/E-1 (27).jpg
./LSTM_PIC/bend/0/E-1 (28).jpg
./LSTM_PIC/bend/0/E-1 (29).jpg
./LSTM_PIC/bend/0/E-1 (3).jpg
./LSTM_PIC/bend/0/E-1 (30).jpg
./LSTM_PIC/bend/0/E-1 (4).jpg
./LSTM_PIC/bend/0/E-1 (5).jpg
./LSTM_PIC/bend/0/E-1 (6).jpg
./LSTM_PIC/bend/0/E-1 (7).jpg
./LSTM_PIC/bend/0/E-1 (8).jpg
./LSTM_PIC/bend/0/E-1 (9).jpg
./LSTM_PIC/bend/1/O-1 (1).jpg
./LSTM_PIC/bend/1/O-1 (10).jpg
./LSTM_PIC/bend/1/

In [34]:
# 전처리, 특성, 라벨 생성
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [35]:
# actions 메핑하기 
label_map = {label:num for num, label in enumerate(actions)}
label_map


{'bend': 0, 'headup': 1, 'right': 2, 'waistup': 3}

In [36]:
np.load("LSTM_DATAS/bend/0/f('bend', 0).npy")

array([137.22032547, 140.01979451, -28.        ])

In [37]:
np.load(f"{data_key}/{action}/{sequence}/f('{action}', {frame_num}).npy")

array([ 173.78558261,  111.36687222, -113.        ])

In [38]:
sequences, labels = [],[]
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequences_length):
            res = np.load(f"{data_key}/{action}/{sequence}/f('{action}', {frame_num}).npy")
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
        

In [39]:
np.array(sequences).shape
# action(4) * 10개의 폴더 , 30개의 사진 , 3개의 각도

(40, 30, 3)

In [40]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [43]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [44]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,3)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [45]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [46]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
2/2 [==============================] - 3s 405ms/step - loss: 159.4067 - categorical_accuracy: 0.2368
Epoch 2/2000
2/2 [==============================] - 1s 386ms/step - loss: 32.8712 - categorical_accuracy: 0.2632
Epoch 3/2000
2/2 [==============================] - 1s 289ms/step - loss: 30.8495 - categorical_accuracy: 0.3684
Epoch 4/2000
2/2 [==============================] - 1s 339ms/step - loss: 33.0871 - categorical_accuracy: 0.2105
Epoch 5/2000
2/2 [==============================] - 1s 272ms/step - loss: 31.4215 - categorical_accuracy: 0.2632
Epoch 6/2000
2/2 [==============================] - 1s 311ms/step - loss: 25.9466 - categorical_accuracy: 0.1579
Epoch 7/2000
2/2 [==============================] - 1s 343ms/step - loss: 30.4060 - categorical_accuracy: 0.2368
Epoch 8/2000
2/2 [==============================] - 1s 282ms/step - loss: 32.6087 - categorical_accuracy: 0.2105
Epoch 9/2000
2/2 [==============================] - 1s 281ms/step - loss: 39.5330 - categorical

In [48]:
r= model.predict(X_test)
r.argmax()

1/1 [==============================] - 0s 67ms/step


2